In [1]:
import pandas as pd
import statistics

In [2]:
coordinates = pd.read_excel('координаты скв.xlsx')
coordinates.head()

,№ скважины,Координата X,Координата Y,Координата забоя Х (по траектории),Координата забоя Y (по траектории)
0,103,482574,6770110,482539.0,6770073.0
1,103ПА,483938,6771248,483951.0,6771257.0
2,1084,478410,6770651,478410.0,6770626.0
3,1085,478791,6771402,478594.0,6770936.0
4,1086,478995,6772092,479215.0,6771562.0


In [3]:
#вычисление длины ствола гс скважины
coordinates['Length'] = coordinates.apply(
    lambda x:((x['Координата забоя Х (по траектории)']-x['Координата X'])**(2)+
              (x['Координата забоя Y (по траектории)']-x['Координата Y'])**(2)) **(0.5)
    , axis=1)
coordinates.head()

,№ скважины,Координата X,Координата Y,Координата забоя Х (по траектории),Координата забоя Y (по траектории),Length
0,103,482574,6770110,482539.0,6770073.0,50.931326
1,103ПА,483938,6771248,483951.0,6771257.0,15.811388
2,1084,478410,6770651,478410.0,6770626.0,25.000000
3,1085,478791,6771402,478594.0,6770936.0,505.929837
4,1086,478995,6772092,479215.0,6771562.0,573.846669


In [4]:
grp=pd.read_excel('фраклист.xlsx',header=[1], index_col=[1])
grp.reset_index(drop=True)
grp.reset_index(inplace=True)
grp=grp.drop(['Unnamed: 0','Подрядчик','Комментарии','Страница','Unnamed: 6','Азимут'],axis=1)
grp=grp.drop(['V гель','V без под','V под','М бр','Гель','Конц','Расход','Эфф','Рпл Хорнер','Проппант','k','Ноб','Нэф','Pпл','Верх','Низ'],axis=1)
grp=grp.drop(['Xf','Hf','Wf','PIdsg','Скин-фактор','JD','FCD','М пр','Покр','План'],axis=1)

grp

C:\Users\USER\miniconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,Номер скважины,Дата,Пласт,Наклон
0,931,2006-07-12,Ю1(1),2.12
1,931,2004-10-05,Ю1(1),NaN
2,913,2015-03-22,Б7,26.00
3,911,2015-05-26,Б7,26.00
4,903,2015-05-11,Б6-7,1.00
...,...,...,...,...
86,277,2003-12-29,Ю1(1),NaN
87,252,2004-08-30,Ю1(1),NaN
88,149,2005-03-31,Ю1(1),NaN
89,1338,2020-09-08,Ю1(1),13.00


In [5]:
#координаты скважин для нашего объекта 
my_wells=grp['Номер скважины'].tolist()
my_wells_no_repeat = [] 
for x in my_wells:
    if x not in my_wells_no_repeat:
        my_wells_no_repeat.append(x)
coordinates=coordinates[coordinates['№ скважины'].isin(my_wells_no_repeat)].reset_index(drop=True)
coordinates=coordinates.drop(['Координата X','Координата Y','Координата забоя Х (по траектории)','Координата забоя Y (по траектории)'],axis=1)
new=coordinates['№ скважины'].tolist()
coordinates['Номер скважины']=new
coordinates=coordinates.drop(['№ скважины'],axis=1)
coordinates

,Length,Номер скважины
0,75.471849,1330
1,10.000000,1338
2,4.123106,149
3,20.000000,252
4,3.605551,277
5,37.161808,300
6,5.385165,340
7,249.369204,383
8,358.788238,409
9,8.062258,418


In [6]:
#понимаем какие скважины горизонтальные по углу наклона

final = grp.merge(coordinates, on='Номер скважины')
final['Длиина ГС, м'] = final.apply(
    lambda x: 
    x['Length'] if x['Наклон']>=70
    else 0, axis=1)
final=final.drop(['Наклон','Length'],axis=1)
final_number = final.groupby('Номер скважины').agg(
    {'Дата': lambda x: 
     x.tolist()}
)

final

,Номер скважины,Дата,Пласт,"Длиина ГС, м"
0,931,2006-07-12,Ю1(1),0.0
1,931,2004-10-05,Ю1(1),0.0
2,913,2015-03-22,Б7,0.0
3,911,2015-05-26,Б7,0.0
4,903,2015-05-11,Б6-7,0.0
...,...,...,...,...
86,277,2003-12-29,Ю1(1),0.0
87,252,2004-08-30,Ю1(1),0.0
88,149,2005-03-31,Ю1(1),0.0
89,1338,2020-09-08,Ю1(1),0.0


In [7]:
#обработка и чтение гтм
grp_1=pd.read_excel('ГТМ.xlsx',header=[1], index_col=[1])
grp_1.reset_index(drop=True)
grp_1.reset_index(inplace=True)
grp_1=grp_1[grp_1['Скважина'].isin(my_wells_no_repeat)].reset_index(drop=True)
grp_1=grp_1.drop(['С начала мероприятия','С начала года','За текущий месяц','P нас','В-ть жидк.','Коб'],axis=1)
grp_1=grp_1.drop(['Пл-ть нефти','Факт','План','Комментарий','Dэк','Пласт','Страница'],axis=1)
grp_1=grp_1.drop(['Unnamed: 11','Hэфф','Pпл.2','H дин.1','Pзаб.2','Состояние.1','Кпр'],axis=1)
grp_1=grp_1.drop(['Скин-фактор.1','Интервалы перф..1','Насос.1','Напор.1','Производительность.1'],axis=1)
grp_1=grp_1.drop(['%.2','Qж.2','Qн.2','Hспуск.1','Кач.1','Част.1','Lхода.1','Метод.1','Скин-фактор','Pзаб.1','Метод','Re'],axis=1)
grp_1=grp_1.drop(['Pпл.1','%.1','Qж.1','Qн.1','Окончание','Начало','Производительность','Насос'],axis=1)
grp_1=grp_1.drop(['Интервалы перф.','Hвд','Состояние','Pзаб','Pпл','H дин','Lхода','Напор'],axis=1)
grp_1=grp_1.drop(['Месторождение','Unnamed: 0','Куст','Част','Кач','Hспуск','Qн','Qж','%'],axis=1)
grp_1

C:\Users\USER\miniconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,Скважина,Тип,Объект разработки до ГТМ,gtmTypeFullName,Краткое описание мероприятий,ВНР,Начало.1,Окончание.1,ВНР.1
0,1330,ВНС,Ю1(1),Ввод новых скважин,Ввод добывающих сква,NaN,2019-09-14,2019-09-24,2019-09-24
1,1330,Переход или приобщение,Ю1(1),Переход на другой объект или приобщение,КР4 Переход на други,NaN,2019-10-11,2019-10-22,2019-10-22
2,1330,ГРП,Ю1(1),Гидравлический разрыв пласта,КР7-2 Проведение ГРП,NaN,2020-01-01,2020-01-20,2020-01-20
3,1330,ОПЗ,Ю1(1),Обработка призабойной зоны скважины,КР7-34 Интенсификаци,NaN,2020-02-03,2020-02-19,2020-02-19
4,1330,Переход или приобщение,Ю1(1),Переход на другой объект или приобщение,КР4 Переход на други,NaN,2020-02-03,2020-02-19,2020-02-19
...,...,...,...,...,...,...,...,...,...
928,931,См_ПО,NaN,Смена насоса,ТР4-1 Смена ШГН,NaN,2013-04-08,2013-04-11,2013-04-11
929,931,Запуск без ремонта,Ю1(1),Запуск без ремонта,Запуск в работу без,NaN,2019-12-30,2019-12-31,2019-12-31
930,931,Запуск без ремонта,Ю1(1),Запуск без ремонта,Запуск в работу без,NaN,2020-02-20,2020-02-20,2020-02-20
931,931,Запуск без ремонта,Ю1(1),Запуск без ремонта,Запуск в работу без,NaN,2020-03-13,2020-03-13,2020-03-13


In [8]:
#даты введения ГРП и ВНС на скважинах очень важно!!!!!!!
grp_3=grp_1
grp_3['Дата ВНР после ГС \\ ГРП \\ЗБГС'] = grp_3.apply(
    lambda x: 
    x['Начало.1'] if 'КР7-2' in str(x['Краткое описание мероприятий'])
    else x['Начало.1'] if 'Ввод добывающих сква' in str(x['Краткое описание мероприятий'])
    else 'нет', axis=1)
grp_3['Тип ГТМ'] = grp_3.apply(
    lambda x: 
    'ГРП' if 'КР7-2' in str(x['Краткое описание мероприятий'])
    else 'ВНС' if 'Ввод добывающих сква' in str(x['Краткое описание мероприятий'])
    else 'нет', axis=1)
grp_3 = grp_3.drop(grp_1[grp_1['Дата ВНР после ГС \\ ГРП \\ЗБГС'] == 'нет'].index)
grp_3 = grp_3.groupby(by=['Скважина','Дата ВНР после ГС \ ГРП \ЗБГС']).agg(
    {'Тип ГТМ': lambda x: 
     x.tolist()[0]}
)
grp_3.reset_index(inplace=True)
vvv=grp_3['Скважина'].tolist()

grp_3['Номер скважины']=vvv
grp_3=grp_3.drop(['Скважина'],axis=1)
grp_3

,Дата ВНР после ГС \ ГРП \ЗБГС,Тип ГТМ,Номер скважины
0,2019-09-14,ВНС,1330
1,2020-01-01,ГРП,1330
2,2020-09-08,ГРП,1338
3,2020-09-13,ВНС,1338
4,2005-03-31,ГРП,149
...,...,...,...
97,2015-04-07,ГРП,913
98,2001-06-05,ВНС,931
99,2001-06-07,ГРП,931
100,2004-10-05,ГРП,931


In [9]:
#число стадий ГРП по гтм
grp_2=grp_1
grp_2['ГРП'] = grp_2.apply(
    lambda x: 
    'грп' if 'КР7-2' in str(x['Краткое описание мероприятий'])
    else 'нет', axis=1)
grp_2 = grp_2.drop(grp_2[grp_2['ГРП'] != 'грп'].index)
grp_2 = grp_2.groupby('Скважина').agg(
    {'ГРП': lambda x: 
     x.tolist()}
)
grp_2.reset_index(inplace=True)
new_1=grp_2['Скважина'].tolist()
grp_2['Номер скважины']=new_1
grp_2=grp_2.drop(['Скважина'],axis=1)
final = final.merge(grp_2, on='Номер скважины')
final['Число стадий ГРП'] = final.apply(
    lambda x: 
    len(x['ГРП']) if len(x['ГРП'])>0
    else 0, axis=1)
final=final.drop(['ГРП'],axis=1)
#число стадий грп по фраклисту
final=final.drop(['Дата'],axis=1)
final_re = final.groupby(by=['Номер скважины']).agg(
    {'Число стадий ГРП': lambda x: 
     statistics.mean(x)}
)
final_re.reset_index(inplace=True)

 
#final_re['Число стадий ГРП'] = final_re.apply(
 #   lambda x: 
  #  1 if x['Длиина ГС, м']==0 and x['Число стадий ГРП']>=1 
   # else x['Число стадий ГРП'], axis=1)
final_re

,Номер скважины,Число стадий ГРП
0,1330,1
1,1338,1
2,149,1
3,252,2
4,277,1
5,300,2
6,340,3
7,383,4
8,409,2
9,418,1


In [10]:
final_go = final.groupby(by=['Номер скважины']).agg(
    {'Длиина ГС, м': lambda x: 
     statistics.mean(x)}
)
final_go.reset_index(inplace=True)
final_go

,Номер скважины,"Длиина ГС, м"
0,1330,0.000000
1,1338,0.000000
2,149,0.000000
3,252,0.000000
4,277,0.000000
5,300,0.000000
6,340,0.000000
7,383,249.369204
8,409,358.788238
9,418,0.000000


In [11]:
final_re = final_re.merge(final_go, on='Номер скважины')
final_re

,Номер скважины,Число стадий ГРП,"Длиина ГС, м"
0,1330,1,0.000000
1,1338,1,0.000000
2,149,1,0.000000
3,252,2,0.000000
4,277,1,0.000000
5,300,2,0.000000
6,340,3,0.000000
7,383,4,249.369204
8,409,2,358.788238
9,418,1,0.000000


In [12]:
final_re['Число стадий ГРП'] = final_re.apply(
    lambda x: 
    1 if x['Длиина ГС, м']==0 and x['Число стадий ГРП']>=1 
    else x['Число стадий ГРП'], axis=1)
final_re

,Номер скважины,Число стадий ГРП,"Длиина ГС, м"
0,1330,1,0.000000
1,1338,1,0.000000
2,149,1,0.000000
3,252,1,0.000000
4,277,1,0.000000
5,300,1,0.000000
6,340,1,0.000000
7,383,4,249.369204
8,409,2,358.788238
9,418,1,0.000000


In [13]:
final_re1=final_re
final_re1 = final_re1.drop(final_re1[final_re1['Длиина ГС, м'] == 0].index).reset_index(drop=True)
spisok=final_re1['Номер скважины'].tolist()
final_re1

,Номер скважины,Число стадий ГРП,"Длиина ГС, м"
0,383,4,249.369204
1,409,2,358.788238
2,480,3,220.975293
3,486,2,244.754244
4,63,1,530.811643
5,632,8,252.878934
6,65,1,706.543700
7,659,3,263.696796


In [14]:
#берем число строк из фраклиста для скважины
final = final.groupby('Номер скважины').agg(
    {'Число стадий ГРП': lambda x: 
     x.tolist()}
)

final['Количество'] = final.apply(
    lambda x: 
    len(x['Число стадий ГРП']) 
    ,axis=1)
final.reset_index(inplace=True)
final=final[final['Номер скважины'].isin(spisok)].reset_index(drop=True)
final=final.drop(['Число стадий ГРП'],axis=1)
final

,Номер скважины,Количество
0,383,3
1,409,4
2,480,5
3,486,5
4,63,9
5,632,7
6,65,8
7,659,5


In [15]:
final_re2 = final_re.merge(final, on='Номер скважины')
final_re2=final_re2.drop(['Число стадий ГРП'],axis=1)
cv=final_re2['Количество'].tolist()
final_re2['Число стадий ГРП']=cv
final_re2=final_re2.drop(['Количество'],axis=1)
final_re2  

,Номер скважины,"Длиина ГС, м",Число стадий ГРП
0,383,249.369204,3
1,409,358.788238,4
2,480,220.975293,5
3,486,244.754244,5
4,63,530.811643,9
5,632,252.878934,7
6,65,706.543700,8
7,659,263.696796,5


In [16]:
final_re=final_re[~final_re['Номер скважины'].isin(spisok)].reset_index(drop=True)
final_re

,Номер скважины,Число стадий ГРП,"Длиина ГС, м"
0,1330,1,0.0
1,1338,1,0.0
2,149,1,0.0
3,252,1,0.0
4,277,1,0.0
5,300,1,0.0
6,340,1,0.0
7,418,1,0.0
8,431,1,0.0
9,439,1,0.0


In [17]:
new_file=pd.concat([final_re,final_re2])
new_file

,Номер скважины,Число стадий ГРП,"Длиина ГС, м"
0,1330,1,0.000000
1,1338,1,0.000000
2,149,1,0.000000
3,252,1,0.000000
4,277,1,0.000000
5,300,1,0.000000
6,340,1,0.000000
7,418,1,0.000000
8,431,1,0.000000
9,439,1,0.000000


In [18]:
grp = grp.groupby('Номер скважины').agg(
    {'Пласт': lambda x: 
     x.tolist()[0]}
)
grp.reset_index(inplace=True)
grp

,Номер скважины,Пласт
0,1330,Ю1(1)
1,1338,Ю1(1)
2,149,Ю1(1)
3,252,Ю1(1)
4,277,Ю1(1)
5,300,Ю1(1)
6,340,Ю1(1)
7,383,Ю1(1)
8,409,Ю1(1)
9,418,Ю1(1)


In [19]:
new_file = new_file.merge(grp, on='Номер скважины')
new_file

,Номер скважины,Число стадий ГРП,"Длиина ГС, м",Пласт
0,1330,1,0.000000,Ю1(1)
1,1338,1,0.000000,Ю1(1)
2,149,1,0.000000,Ю1(1)
3,252,1,0.000000,Ю1(1)
4,277,1,0.000000,Ю1(1)
5,300,1,0.000000,Ю1(1)
6,340,1,0.000000,Ю1(1)
7,418,1,0.000000,Ю1(1)
8,431,1,0.000000,Ю1(1)
9,439,1,0.000000,Ю1(1)


In [20]:
new_file['Полудлина трещины, м'] = new_file.apply(
    lambda x: 
    100, axis=1)
new_file['Ширина трещины, мм'] = new_file.apply(
    lambda x: 
    4, axis=1)
new_file['Проницаемость проппанта, Д'] = new_file.apply(
    lambda x: 
    100, axis=1)
new_file

,Номер скважины,Число стадий ГРП,"Длиина ГС, м",Пласт,"Полудлина трещины, м","Ширина трещины, мм","Проницаемость проппанта, Д"
0,1330,1,0.000000,Ю1(1),100,4,100
1,1338,1,0.000000,Ю1(1),100,4,100
2,149,1,0.000000,Ю1(1),100,4,100
3,252,1,0.000000,Ю1(1),100,4,100
4,277,1,0.000000,Ю1(1),100,4,100
5,300,1,0.000000,Ю1(1),100,4,100
6,340,1,0.000000,Ю1(1),100,4,100
7,418,1,0.000000,Ю1(1),100,4,100
8,431,1,0.000000,Ю1(1),100,4,100
9,439,1,0.000000,Ю1(1),100,4,100


In [21]:
#объединение грп и нью файла по скважинам
sotired = grp_3.merge(new_file, on='Номер скважины')
sotired

,Дата ВНР после ГС \ ГРП \ЗБГС,Тип ГТМ,Номер скважины,Число стадий ГРП,"Длиина ГС, м",Пласт,"Полудлина трещины, м","Ширина трещины, мм","Проницаемость проппанта, Д"
0,2019-09-14,ВНС,1330,1,0.0,Ю1(1),100,4,100
1,2020-01-01,ГРП,1330,1,0.0,Ю1(1),100,4,100
2,2020-09-08,ГРП,1338,1,0.0,Ю1(1),100,4,100
3,2020-09-13,ВНС,1338,1,0.0,Ю1(1),100,4,100
4,2005-03-31,ГРП,149,1,0.0,Ю1(1),100,4,100
...,...,...,...,...,...,...,...,...,...
97,2015-04-07,ГРП,913,1,0.0,Б7,100,4,100
98,2001-06-05,ВНС,931,1,0.0,Ю1(1),100,4,100
99,2001-06-07,ГРП,931,1,0.0,Ю1(1),100,4,100
100,2004-10-05,ГРП,931,1,0.0,Ю1(1),100,4,100


In [22]:
#добавляем последнй столбец
sotired['ГС/ННС'] = sotired.apply(
    lambda x: 
    'ННС' if x['Длиина ГС, м']==0
    else 'ГС', axis=1)
sotired

,Дата ВНР после ГС \ ГРП \ЗБГС,Тип ГТМ,Номер скважины,Число стадий ГРП,"Длиина ГС, м",Пласт,"Полудлина трещины, м","Ширина трещины, мм","Проницаемость проппанта, Д",ГС/ННС
0,2019-09-14,ВНС,1330,1,0.0,Ю1(1),100,4,100,ННС
1,2020-01-01,ГРП,1330,1,0.0,Ю1(1),100,4,100,ННС
2,2020-09-08,ГРП,1338,1,0.0,Ю1(1),100,4,100,ННС
3,2020-09-13,ВНС,1338,1,0.0,Ю1(1),100,4,100,ННС
4,2005-03-31,ГРП,149,1,0.0,Ю1(1),100,4,100,ННС
...,...,...,...,...,...,...,...,...,...,...
97,2015-04-07,ГРП,913,1,0.0,Б7,100,4,100,ННС
98,2001-06-05,ВНС,931,1,0.0,Ю1(1),100,4,100,ННС
99,2001-06-07,ГРП,931,1,0.0,Ю1(1),100,4,100,ННС
100,2004-10-05,ГРП,931,1,0.0,Ю1(1),100,4,100,ННС


In [23]:
#редактируем его в зависимости от наличия грп
sotired['ГС/ННС'] = sotired.apply(
    lambda x: 
    str(x['ГС/ННС'])+'+ГРП' if str(x['Тип ГТМ'])=='ГРП'
    else x['ГС/ННС'], axis=1)
sotired

,Дата ВНР после ГС \ ГРП \ЗБГС,Тип ГТМ,Номер скважины,Число стадий ГРП,"Длиина ГС, м",Пласт,"Полудлина трещины, м","Ширина трещины, мм","Проницаемость проппанта, Д",ГС/ННС
0,2019-09-14,ВНС,1330,1,0.0,Ю1(1),100,4,100,ННС
1,2020-01-01,ГРП,1330,1,0.0,Ю1(1),100,4,100,ННС+ГРП
2,2020-09-08,ГРП,1338,1,0.0,Ю1(1),100,4,100,ННС+ГРП
3,2020-09-13,ВНС,1338,1,0.0,Ю1(1),100,4,100,ННС
4,2005-03-31,ГРП,149,1,0.0,Ю1(1),100,4,100,ННС+ГРП
...,...,...,...,...,...,...,...,...,...,...
97,2015-04-07,ГРП,913,1,0.0,Б7,100,4,100,ННС+ГРП
98,2001-06-05,ВНС,931,1,0.0,Ю1(1),100,4,100,ННС
99,2001-06-07,ГРП,931,1,0.0,Ю1(1),100,4,100,ННС+ГРП
100,2004-10-05,ГРП,931,1,0.0,Ю1(1),100,4,100,ННС+ГРП


In [24]:
#перемещаем столбцы в нужном порядке
r = sotired.pop('Дата ВНР после ГС \ ГРП \ЗБГС')
sotired1 = pd.concat([sotired, r], 1)


C:\Users\USER\miniconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


In [25]:
#перемещаем столбцы в нужном порядке
q = sotired1.pop('ГС/ННС')
sotired2 = pd.concat([sotired1, q], 1)


C:\Users\USER\miniconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


In [26]:
#перемещаем столбцы в нужном порядке
z = sotired2.pop('Тип ГТМ')
sotired3 = pd.concat([sotired2, z], 1)


C:\Users\USER\miniconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


In [27]:
#перемещаем столбцы в нужном порядке
d = sotired3.pop('Пласт')
sotired4 = pd.concat([sotired3, d], 1)
i=sotired4['Номер скважины'].tolist()
sotired4['Скважина №']=i
sotired4=sotired4.drop(['Номер скважины'],axis=1)


C:\Users\USER\miniconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


In [28]:
sotired_final = sotired4.reindex(columns=['Скважина №', 'Длиина ГС, м', 'Число стадий ГРП','Полудлина трещины, м','Ширина трещины, мм','Проницаемость проппанта, Д','Дата ВНР после ГС \ ГРП \ЗБГС','ГС/ННС','Тип ГТМ','Пласт'])
sotired_final

,Скважина №,"Длиина ГС, м",Число стадий ГРП,"Полудлина трещины, м","Ширина трещины, мм","Проницаемость проппанта, Д",Дата ВНР после ГС \ ГРП \ЗБГС,ГС/ННС,Тип ГТМ,Пласт
0,1330,0.0,1,100,4,100,2019-09-14,ННС,ВНС,Ю1(1)
1,1330,0.0,1,100,4,100,2020-01-01,ННС+ГРП,ГРП,Ю1(1)
2,1338,0.0,1,100,4,100,2020-09-08,ННС+ГРП,ГРП,Ю1(1)
3,1338,0.0,1,100,4,100,2020-09-13,ННС,ВНС,Ю1(1)
4,149,0.0,1,100,4,100,2005-03-31,ННС+ГРП,ГРП,Ю1(1)
...,...,...,...,...,...,...,...,...,...,...
97,913,0.0,1,100,4,100,2015-04-07,ННС+ГРП,ГРП,Б7
98,931,0.0,1,100,4,100,2001-06-05,ННС,ВНС,Ю1(1)
99,931,0.0,1,100,4,100,2001-06-07,ННС+ГРП,ГРП,Ю1(1)
100,931,0.0,1,100,4,100,2004-10-05,ННС+ГРП,ГРП,Ю1(1)


In [29]:
sotired_final['Число стадий ГРП'] = sotired.apply(
    lambda x: 
    0 if str(x['Тип ГТМ'])=='ВНС'
    else x['Число стадий ГРП'], axis=1)
sotired_final

,Скважина №,"Длиина ГС, м",Число стадий ГРП,"Полудлина трещины, м","Ширина трещины, мм","Проницаемость проппанта, Д",Дата ВНР после ГС \ ГРП \ЗБГС,ГС/ННС,Тип ГТМ,Пласт
0,1330,0.0,0,100,4,100,2019-09-14,ННС,ВНС,Ю1(1)
1,1330,0.0,1,100,4,100,2020-01-01,ННС+ГРП,ГРП,Ю1(1)
2,1338,0.0,1,100,4,100,2020-09-08,ННС+ГРП,ГРП,Ю1(1)
3,1338,0.0,0,100,4,100,2020-09-13,ННС,ВНС,Ю1(1)
4,149,0.0,1,100,4,100,2005-03-31,ННС+ГРП,ГРП,Ю1(1)
...,...,...,...,...,...,...,...,...,...,...
97,913,0.0,1,100,4,100,2015-04-07,ННС+ГРП,ГРП,Б7
98,931,0.0,0,100,4,100,2001-06-05,ННС,ВНС,Ю1(1)
99,931,0.0,1,100,4,100,2001-06-07,ННС+ГРП,ГРП,Ю1(1)
100,931,0.0,1,100,4,100,2004-10-05,ННС+ГРП,ГРП,Ю1(1)


In [38]:
n=1
z=sotired_final['Пласт'].value_counts()[:n].index.tolist()
plast=str(z[0])
plast

'Ю1(1)'

In [30]:
sotired_final.to_excel('GRP_redacted.xlsx')